Voy a leer un pdf y crear una coleccion en chromadb de cada pedazo. Hago similarity seach en esa coleccion y me quedo con el primero que espero porfavo diosito que sea el correcto. 

El esquema de trabajo seria: 

Leo pdf y divido en pedazos `data_loader.py` -> hago similarity search para escoger el pedazo **(este notebook)** `aun no existe.py` -> extarigo datos `data_extractors.py`

In [2]:
from tools.data_loader import PDFReader

pdf = 'licitaciones_filtered_pdfs/licitaciones/licitaciones_filtered_pdfs/contratacion/BOE-B-2015-17810.pdf'
pdf_reader = PDFReader(pdf, chunk_size=600, overlap=50)
print(len(pdf_reader.document_parts))


10


chromadb

In [2]:
import chromadb

client = chromadb.Client()

In [3]:
try:
    client.delete_collection(name=pdf_reader.filename)
except:
    pass

In [4]:
collection = client.create_collection(name=pdf_reader.filename, metadata={"hnsw:space": "cosine"})

conseguimos embeddings y anadimos a la collection

In [5]:
import ollama
def get_embedding(text):
    embedding_dict = ollama.embeddings(model='llama3', prompt=text)
    vector = embedding_dict['embedding']
    return vector

In [6]:
i = 0
for part in pdf_reader.document_parts:
    i += 1
    text = part.text
    vector = get_embedding(text)
    collection.add(
        embeddings=[vector],
        documents=[text],
        metadatas=[part.metadata],
        ids=[str(i)]
    )

In [7]:
query_text = "Cantidad monetaria total de la licitacion"
query_vector = get_embedding(query_text)

results = collection.query(
    query_embeddings=[query_vector],
    n_results=3
)
results

{'ids': [['2', '7', '9']],
 'distances': [[0.5897547602653503, 0.643503725528717, 0.6634917259216309]],
 'metadatas': [[{'filename': 'BOE-B-2015-17810.pdf', 'page': 1},
   {'filename': 'BOE-B-2015-17810.pdf', 'page': 2},
   {'filename': 'BOE-B-2015-17810.pdf', 'page': 2}]],
 'embeddings': None,
 'documents': [['te a la\nverificación  de  ayudas  cofinanciadas  con  FEDER  de  diversas\nconvocatorias  de  proyectos  de  I+D.  Expediente:  J15.014.13.\n1. Entidad adjudicadora: Datos generales y datos para la obtención de la\ninformación:\na)  Organismo:  Junta  de  Contratación  del  Ministerio  de  Economía  y\nCompetitividad.\nb) Dependencia que tramita el expediente: Junta de Contratación del Ministerio\nde Economía y Competitividad.\nc) Obtención de documentación e información:\n1) Dependencia: Junta de Contratación del Ministerio de Economía y\nCompetitividad',
   '785,12 euros. Importe total: 370.000,00 euros.\n6. Garantías exigidas. Definitiva (%): 5.\n7. Requisitos específicos de

In [8]:
print(pdf_reader.get_part(2).text) # No tiene la cantidad total del contrato
print("-----------------------------------------------------------------")
print(pdf_reader.get_part(7).text) # No tiene la cantidad total del contrato
print("-----------------------------------------------------------------")
print(pdf_reader.get_part(9).text) # Tiene la cantidad total del contrato

te a la
verificación  de  ayudas  cofinanciadas  con  FEDER  de  diversas
convocatorias  de  proyectos  de  I+D.  Expediente:  J15.014.13.
1. Entidad adjudicadora: Datos generales y datos para la obtención de la
información:
a)  Organismo:  Junta  de  Contratación  del  Ministerio  de  Economía  y
Competitividad.
b) Dependencia que tramita el expediente: Junta de Contratación del Ministerio
de Economía y Competitividad.
c) Obtención de documentación e información:
1) Dependencia: Junta de Contratación del Ministerio de Economía y
Competitividad
-----------------------------------------------------------------
785,12 euros. Importe total: 370.000,00 euros.
6. Garantías exigidas. Definitiva (%): 5.
7. Requisitos específicos del contratista:
b) Solvencia económica y financiera y solvencia técnica y profesional: Solvencia
económica y financiera: Cifra anual de negocio (Todos los licitadores ver
casilla 16 del cuadro resumen del PCAP). Solvencia técnica y profesional:
Trabajos realizados (T

Probamos el nuevo modulo

In [4]:
from tools.similarity_search import EmbeddingsDB

prueba = EmbeddingsDB(pdf_reader=pdf_reader)
prueba.add_documents_to_db()

Collection BOE-B-2015-17810.pdf already exists. Do you want to overwrite it?


In [5]:
prueba.query_documents(query_text="Cantidad monetaria total de la licitacion", n_results=3)

{'ids': [['2', '7', '9']],
 'distances': [[0.5897547602653503, 0.643503725528717, 0.6634917259216309]],
 'metadatas': [[{'filename': 'BOE-B-2015-17810.pdf', 'page': 1},
   {'filename': 'BOE-B-2015-17810.pdf', 'page': 2},
   {'filename': 'BOE-B-2015-17810.pdf', 'page': 2}]],
 'embeddings': None,
 'documents': [['te a la\nverificación  de  ayudas  cofinanciadas  con  FEDER  de  diversas\nconvocatorias  de  proyectos  de  I+D.  Expediente:  J15.014.13.\n1. Entidad adjudicadora: Datos generales y datos para la obtención de la\ninformación:\na)  Organismo:  Junta  de  Contratación  del  Ministerio  de  Economía  y\nCompetitividad.\nb) Dependencia que tramita el expediente: Junta de Contratación del Ministerio\nde Economía y Competitividad.\nc) Obtención de documentación e información:\n1) Dependencia: Junta de Contratación del Ministerio de Economía y\nCompetitividad',
   '785,12 euros. Importe total: 370.000,00 euros.\n6. Garantías exigidas. Definitiva (%): 5.\n7. Requisitos específicos de